# Common Random Variables
Polynomial chaos expansion (PCE) is a Hilbert space technique for random variables with finite variance.
Mathematically equivalent to Fourier series expansions for periodic signals, PCE allows to characterize a random variable in terms of its PCE coefficients (aka Fourier coefficients).
That is, the PCE of a random variable $\mathsf{x}$ is given by
$$
\mathsf{x} = \sum_{i=0}^L x_i \phi_i,
$$
where $x_i$ are the so-called PCE coefficients, and $\phi_i$ are the orthogonal polynomials that are orthogonal relative to the probability density function of $\mathsf{x}$.

This tutorial walks you through the PCE of common random variables, namely Gaussian (`gaussian`), Beta (`beta01`), Uniform(`uniform01`), Logistic (`logistic`), and shows how they are implemented in `PolyChaos`.

## Construction of Basis

We begin by specifying the names and, if any, parameters for the uncertainties.

In [ ]:
using Revise
using PolyChaos
α, β = 1.3, 2.2
polynames = ["gaussian", "beta01", "uniform01", "logistic"]
pars = [Dict(), Dict(:shape_a=>α,:shape_b=>β), Dict(), Dict()]

The orthogonal polynomials are constructed using `OrthoPoly` (here of degree at most `d`, and stored in the dictionary `myops`).

In [ ]:
d = 6
myops = Dict()
for (i,name) in enumerate(polynames)
    myops[name]=OrthoPoly(name,d,pars[i])
end

For example, let's evaluate the Gaussian basis polynomials at some points

In [ ]:
points, degrees = randn(10), 0:2:d
op_gauss=myops["gaussian"]
[ evaluate(degree,points,op_gauss) for degree in degrees ]

If a quadrature rule is required, this can be added by calling `OrthoPolyQ`

In [ ]:
myopqs = Dict()
for (i,name) in enumerate(names)
    myopqs[name]=OrthoPolyQ(name,d,pars[i])
end

## Finding PCE Coefficients
Having constructed the orthogonal bases, the question remains how to find the PCE coefficients for the common random variables.
Every random variable can be characterized exactly by two PCE coefficients.
For a Gaussian random variable, this is familiar: the mean and the variance suffice to describe a Gaussian random variable entirely.
The same is true for any random variable of finite variance given the right basis.
The function `convert2affinePCE` provides the first two PCE coefficients (hence the name affine) for the common random variables.

### Gaussian
Given the Gaussian random variable $\mathsf{x} \sim \mathcal{N}(\mu, \sigma^2)$ with $\sigma > 0$, the affine PCE coefficients are

In [ ]:
# Gaussian
μ, σ = 2., 0.2
pce_gaussian = convert2affinePCE("gaussian",μ,σ)
# Uniform

### Uniform
Given the uniform random variable $\mathsf{x} \sim \mathcal{U}(a, b)$ with finite support $a<b$, the affine PCE coefficients are

In [ ]:
a, b = -0.3, 1.2
convert2affinePCE("uniform01",a,b)

Instead, if the expected value and standard deviation are known, the affine PCE coefficients of the uniform random variable are

In [ ]:
pce_uniform = convert2affinePCE("uniform01",μ,σ;kind=:μσ)
# notice that the zero-order coefficient IS equivalent to the expected value μ

### Beta
Given the Beta random variable $\mathsf{x} \sim \mathcal{B}(a, b, \alpha, \beta)$ with finite support $a<b$ and shape parameters $\alpha, \beta > 0$, the affine PCE coefficients are

In [ ]:
convert2affinePCE("beta01",a,b,Dict(:shape_a=>α,:shape_b=>β))

Instead, if the expected value and standard deviation are known, the affine PCE coefficients of the uniform random variable are

In [ ]:
pce_beta = convert2affinePCE("beta01",μ,σ,Dict(:shape_a=>α,:shape_b=>β); kind=:μσ)

### Logistic

Given the logstic random variable $\mathsf{x} \sim \mathcal{L}(a_1,a_2)$ where $a_2>0$ with the probability density function
$$
\rho(t) = \frac{1}{4 a_2} \, \operatorname{sech}^2 \left(\frac{t-a_1}{2a_2}\right)
$$
the affine PCE coefficients of the uniform random variable are

In [ ]:
a1, a2 = μ, sqrt(3)*σ/pi
pce_logistic = convert2affinePCE("logistic",a1,a2)

## Moments
It is a key feature of PCE to compute moments from the PCE coefficients alone; no sampling is required.

### Gaussian

In [ ]:
mean(pce_gaussian,myops["gaussian"]), std(pce_gaussian,myops["gaussian"])

### Uniform

In [ ]:
mean(pce_uniform,myops["uniform01"]), std(pce_uniform,myops["uniform01"])

### Beta

In [ ]:
mean(pce_beta,myops["beta01"]), std(pce_beta,myops["beta01"])

### Logistic

In [ ]:
mean(pce_logistic,myops["logistic"]), std(pce_logistic,myops["logistic"])

## Sampling
Having found the PCE coefficients, it may be useful to sample the random variables.
That means, find $N$ realizations of the random variable that obey the random variable's probability density function.
This is done in two steps:
1. Draw $N$ samples from the measure (`sampleMeasure()`), and then
2. Evaluate the basis polynomials and multiply times the PCE coefficients, i.e. $\sum_{i=0}^L x_i \phi_i(\xi_j)$ where $\xi_j$ is the $j$-th sample from the measure (`evaluatePCE()`).

Both steps are combined in the function `samplepCE()`.

### Gaussian

In [ ]:
using Statistics
N = 1000
ξ_gaussian = sampleMeasure(N,myops["gaussian"])
samples_gaussian = evaluatePCE(pce_gaussian,ξ_gaussian,myops["gaussian"])
# samplePCE(N,pce_gaussian,myops["gaussian"])

### Uniform

In [ ]:
ξ_uniform = sampleMeasure(N,myops["uniform01"])
samples_uniform = evaluatePCE(pce_uniform,ξ_uniform,myops["uniform01"])
# samples_uniform = samplePCE(N,pce_uniform,myops["uniform01"])

### Beta

In [ ]:
ξ_beta = sampleMeasure(N,myops["beta01"])
samples_beta = evaluatePCE(pce_beta,ξ_beta,myops["beta01"])
# samples_beta = samplePCE(N,pce_beta,myops["beta01"])

### Logistic

In [ ]:
ξ_logistic = sampleMeasure(N,myops["logistic"])
samples_logistic = evaluatePCE(pce_logistic,ξ_logistic,myops["logistic"])
# samples_logistic = samplePCE(N,pce_logistic,myops["logistic"])